# Neural DSL Quick Start Tutorial

Welcome to Neural DSL! This tutorial will guide you through:
1. Installing Neural DSL
2. Creating your first model
3. Compiling and running the model
4. Visualizing the architecture
5. Debugging with NeuralDbg

**Time to complete:** ~15 minutes

**Prerequisites:**
- Python 3.8+
- Basic understanding of neural networks
- pip package manager

## Step 1: Installation

First, let's install Neural DSL. We'll install with TensorFlow backend for this tutorial.

In [ ]:
# Install Neural DSL with TensorFlow
!pip install neural-dsl tensorflow

# Verify installation
import neural
print(f"Neural DSL version: {neural.__version__}")

## Step 2: Create Your First Model

Let's create a simple neural network for MNIST digit classification.

We'll write the model in Neural DSL syntax and save it to a file.

In [ ]:
# Define model in Neural DSL
dsl_code = """
network SimpleClassifier {
  # Input: 28x28 grayscale images (MNIST)
  input: (28, 28, 1)
  
  layers:
    # Convolutional feature extraction
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu")
    MaxPooling2D(pool_size=(2, 2))
    
    # Flatten and classify
    Flatten()
    Dense(units=128, activation="relu")
    Dropout(rate=0.5)
    Output(units=10, activation="softmax")
  
  # Training configuration
  loss: "sparse_categorical_crossentropy"
  optimizer: Adam(learning_rate=0.001)
  metrics: ["accuracy"]
  
  train {
    epochs: 10
    batch_size: 64
    validation_split: 0.2
  }
}
"""

# Save to file
with open('my_first_model.neural', 'w') as f:
    f.write(dsl_code)

print("✅ Model definition saved to 'my_first_model.neural'")

### Understanding the DSL Syntax

Let's break down what each part means:

- **`network SimpleClassifier { ... }`**: Defines a neural network named SimpleClassifier
- **`input: (28, 28, 1)`**: Specifies input shape (height, width, channels)
- **`layers:`**: Lists all layers in sequential order
- **`Conv2D(filters=32, ...)`**: Convolutional layer with 32 filters
- **`Output(units=10, ...)`**: Final layer with 10 classes
- **`loss:`**: Loss function for training
- **`optimizer:`**: Optimization algorithm
- **`train { ... }`**: Training configuration

## Step 3: Compile the Model

Neural DSL can generate code for different backends. Let's compile to TensorFlow:

In [ ]:
# Compile DSL to TensorFlow
!neural compile my_first_model.neural --backend tensorflow --output simple_model_tf.py

print("\n✅ Model compiled successfully!")
print("Generated file: simple_model_tf.py")

Let's look at a snippet of the generated code:

In [ ]:
# Read and display first 30 lines of generated code
with open('simple_model_tf.py', 'r') as f:
    lines = f.readlines()[:30]
    print(''.join(lines))
    print("\n... (truncated) ...")

## Step 4: Load and Prepare Data

Before training, let's load the MNIST dataset:

In [ ]:
import tensorflow as tf
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

print(f"Training samples: {x_train.shape[0]}")
print(f"Test samples: {x_test.shape[0]}")
print(f"Input shape: {x_train.shape[1:]}")

Visualize some sample images:

In [ ]:
import matplotlib.pyplot as plt

# Display first 10 images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i].reshape(28, 28), cmap='gray')
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## Step 5: Train the Model

Now let's import and train the generated model:

In [ ]:
# Import the generated model code
import sys
sys.path.insert(0, '.')
from simple_model_tf import create_model

# Create and compile model
model = create_model()

# Display model summary
model.summary()

In [ ]:
# Train the model (just 3 epochs for demo)
history = model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

print("\n✅ Training completed!")

## Step 6: Evaluate and Visualize Results

Let's evaluate the model and visualize training progress:

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss', marker='o')
ax2.plot(history.history['val_loss'], label='Validation Loss', marker='o')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Model Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 7: Make Predictions

Let's test our model on some examples:

In [ ]:
# Make predictions on test set
predictions = model.predict(x_test[:10])
predicted_classes = np.argmax(predictions, axis=1)

# Visualize predictions
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_test[i].reshape(28, 28), cmap='gray')
    true_label = y_test[i]
    pred_label = predicted_classes[i]
    confidence = predictions[i][pred_label]
    
    color = 'green' if true_label == pred_label else 'red'
    ax.set_title(f"True: {true_label}\nPred: {pred_label} ({confidence:.2%})", 
                 color=color)
    ax.axis('off')

plt.tight_layout()
plt.show()

## Step 8: Visualize Model Architecture

Neural DSL can generate architecture diagrams:

In [ ]:
# Generate architecture visualization
!neural visualize my_first_model.neural --format png

print("✅ Architecture diagrams generated!")
print("Files: architecture.png, shape_propagation.html")

## Step 9: Switching Backends

One of Neural DSL's key features is easy backend switching. Let's compile to PyTorch:

In [ ]:
# Compile same model to PyTorch
!neural compile my_first_model.neural --backend pytorch --output simple_model_torch.py

print("\n✅ PyTorch version compiled!")
print("\nThe SAME model definition generated code for both TensorFlow and PyTorch!")

## Step 10: Debugging with NeuralDbg (Optional)

Neural DSL includes a powerful debugging dashboard. To use it:

```bash
# In terminal, run:
neural debug my_first_model.neural

# Then open browser to: http://localhost:8050
```

Features:
- Real-time execution tracing
- Gradient flow visualization
- Dead neuron detection
- Anomaly detection
- Memory and FLOP profiling

## Summary

Congratulations! You've completed the Neural DSL quickstart tutorial. You learned:

✅ How to install Neural DSL  
✅ How to write models in DSL syntax  
✅ How to compile to different backends  
✅ How to train and evaluate models  
✅ How to visualize architectures  
✅ How to switch between TensorFlow and PyTorch  

## Next Steps

Continue learning with these tutorials:

1. **[Hyperparameter Optimization Tutorial](hpo_tutorial.ipynb)** - Learn to optimize your models
2. **[Advanced Architectures Tutorial](advanced_architectures.ipynb)** - Build complex models
3. **[Cloud Integration Tutorial](cloud_tutorial.ipynb)** - Run on Kaggle, Colab, AWS
4. **[Debugging Tutorial](debugging_tutorial.ipynb)** - Master NeuralDbg

## Resources

- **Documentation:** [docs/](../../docs/)
- **Examples:** [examples/](../../examples/)
- **Discord:** [Join our community](https://discord.gg/KFku4KvS)
- **GitHub:** [Issues & Discussions](https://github.com/Lemniscate-world/Neural)

Happy modeling! 🚀